<a href="https://colab.research.google.com/github/satuelisa/DBforWeb/blob/main/DBW_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/satuelisa/DBforWeb/main/joins/students.csv' # new example data
students = pd.read_csv(url)
url = 'https://raw.githubusercontent.com/satuelisa/DBforWeb/main/joins/grades.csv' # today we need more than one table
grades = pd.read_csv(url)

First and foremost, it is best go take a look at the two tables: we have info on [students](https://github.com/satuelisa/DBforWeb/blob/main/joins/students.csv) (including the program they are enrolled in, assuming a single program per student) and, on the other hand, on [grades](https://github.com/satuelisa/DBforWeb/blob/main/joins/grades.csv).

In [2]:
import sqlite3
connector = sqlite3.connect('example.db')
cursor = connector.cursor()

In [3]:
# first table
fields = ','.join(students.columns)
cursor.execute(f'CREATE TABLE students ({fields});') 
rows = students.to_dict(orient = 'split')['data']
wildcard = ('?,' * len(students.columns))[:-1]
cursor.executemany(f'INSERT INTO students ({fields}) VALUES ({wildcard});', rows) 

In [4]:
# second table
fields = ','.join(grades.columns)
cursor.execute(f'CREATE TABLE grades ({fields});') 
rows = grades.to_dict(orient = 'split')['data']
wildcard = ('?,' * len(grades.columns))[:-1]
cursor.executemany(f'INSERT INTO grades ({fields}) VALUES ({wildcard});', rows) 

In [5]:
connector.commit() # send it in
connector.close() # close the connection

Now, if we want to see how a specific program is doing, gradewise, we would need to combine information from the two tables to get the students of that program and then the grades of those students.

In [6]:
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor = connector.execute('SELECT * FROM students JOIN grades ON students.StudentID == grades.StudentID;')
for row in cursor.fetchall():
  print(row)
connector.close()

(123, 'Carmen', 'HR', 123, 'Programming', 82)
(123, 'Carmen', 'HR', 123, 'Stats', 87)
(123, 'Carmen', 'HR', 123, 'Writing', 78)
(125, 'Emilia', 'Finance', 125, 'Writing', 93)
(132, 'Sami', 'IT', 132, 'Programming', 79)
(139, 'Carlo', 'HR', 139, 'Stats', 93)
(141, 'Jules', 'Finance', 141, 'Stats', 78)
(141, 'Jules', 'Finance', 141, 'Writing', 84)
(147, 'James', 'IT', 147, 'Programming', 96)
(151, 'Carmen', 'Finance', 151, 'Programming', 67)
(152, 'Guadalupe', 'IT', 152, 'Writing', 64)
(156, 'Ivan', 'HR', 156, 'Writing', 89)
(163, 'Cecilia', 'IT', 163, 'Programming', 85)
(163, 'Cecilia', 'IT', 163, 'Stats', 73)
(163, 'Cecilia', 'IT', 163, 'Writing', 92)


We could also shove the result into a `pandas` *dataframe* so it looks prettier.

In [7]:
connector = sqlite3.connect('example.db')
result = pd.read_sql_query('SELECT * FROM students JOIN grades ON students.StudentID == grades.StudentID;', connector)
connector.close()
result

,StudentID,Name,Program,StudentID,CourseID,Grade
0,123,Carmen,HR,123,Programming,82
1,123,Carmen,HR,123,Stats,87
2,123,Carmen,HR,123,Writing,78
3,125,Emilia,Finance,125,Writing,93
4,132,Sami,IT,132,Programming,79
5,139,Carlo,HR,139,Stats,93
6,141,Jules,Finance,141,Stats,78
7,141,Jules,Finance,141,Writing,84
8,147,James,IT,147,Programming,96
9,151,Carmen,Finance,151,Programming,67


Note that the `StudentID` column is now duplicated as both tables contained it. Also, if a student has no grade (like `Elisa`), those rows are omitted in the result of this `join` since the second table has no matches for that `StudentID`. How would we go about observing that? We could make a combination of all grades to all students and leave gradeless rows for those who have not completed any courses. This is possible with an *outer* join, but `SQLite` does not support that. This means that we have to vamp up out toolset for this session. However, our next tool does not allow us to easily read a CSV file from an URL, so let's prepare to make the `INSERT` statements here.

In [20]:
for row in students.iterrows():
  sID = row[1]['StudentID']
  sName = row[1]['Name']
  pr = row[1]['Program']
  print(f"INSERT INTO students(StudentID, FirstName, ProgramName) VALUES ({sID}, '{sName}', '{pr}');")

INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (123, 'Carmen', 'HR');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (125, 'Emilia', 'Finance');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (132, 'Sami', 'IT');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (139, 'Carlo', 'HR');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (141, 'Jules', 'Finance');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (147, 'James', 'IT');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (151, 'Carmen', 'Finance');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (152, 'Guadalupe', 'IT');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (156, 'Ivan', 'HR');
INSERT INTO students(StudentID, FirstName, ProgramName) VALUES (163, 'Cecilia', 'IT');


In [22]:
for row in grades.iterrows():
  sID = row[1]['StudentID']
  cName = row[1]['CourseID']
  gr = row[1]['Grade']
  print(f"INSERT INTO grades(StudentID, Course, Grade) VALUES ({sID}, '{cName}', {gr});")

INSERT INTO grades(StudentID, Course, Grade) VALUES (123, 'Stats', 87);
INSERT INTO grades(StudentID, Course, Grade) VALUES (139, 'Stats', 93);
INSERT INTO grades(StudentID, Course, Grade) VALUES (141, 'Stats', 78);
INSERT INTO grades(StudentID, Course, Grade) VALUES (163, 'Stats', 73);
INSERT INTO grades(StudentID, Course, Grade) VALUES (123, 'Programming', 82);
INSERT INTO grades(StudentID, Course, Grade) VALUES (132, 'Programming', 79);
INSERT INTO grades(StudentID, Course, Grade) VALUES (147, 'Programming', 96);
INSERT INTO grades(StudentID, Course, Grade) VALUES (151, 'Programming', 67);
INSERT INTO grades(StudentID, Course, Grade) VALUES (163, 'Programming', 85);
INSERT INTO grades(StudentID, Course, Grade) VALUES (123, 'Writing', 78);
INSERT INTO grades(StudentID, Course, Grade) VALUES (125, 'Writing', 93);
INSERT INTO grades(StudentID, Course, Grade) VALUES (141, 'Writing', 84);
INSERT INTO grades(StudentID, Course, Grade) VALUES (152, 'Writing', 64);
INSERT INTO grades(Student